# **Main script for deforestation prediction**<br/>
checked OS: Google Colab Pro+<br/>

Suguru Kanoga, 16-Aug.-2024<br/>
Artificial Intelligence Research Center, National Institute of Advanced Industrial Science and Technology (AIST)<br/>
E-mail: s.kanouga@aist.go.jp<br/>

Updated points<br/>
1. improved flexibility of whole scripts
2. added config function for managing variables
3. added new feature from road information
4. added regression scenario (but not effective)
5. separated training and evaluation steps
6. added future predicition function
7. added two masking functions (based on output probability and sum of deforestated area in a mesh)
8. added EarlyStopping function
9. added save function for geojson file to visualize results in QGIS software
10. adapted to Google Colab

**check python version**

In [1]:
!python --version

Python 3.10.12


**install packages**

In [2]:
!pip install scikit-learn==1.3.2
!pip install skorch==0.15.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.3/239.3 kB 12.9 MB/s eta 0:00:00


**Mount your Google Drive and set main directory**

In [3]:
# mount your Google Drive
from google.colab import drive

WORK_DRIVE = '/content/drive'
drive.mount(WORK_DRIVE)

# set main directory
import os

WORK_DIR = os.path.join(WORK_DRIVE, 'MyDrive/Recurrent_deep_learning_based_monthly_deforestation_prediction_v2')
os.chdir(WORK_DIR)
!ls

Mounted at /content/drive
config.py    hyperparameter_search.py  network.py  preprocess.py     result
data	     main.py		       predict	   __pycache__	     train.py
evaluate.py  model		       predict.py  requirements.txt  utils.py


**import packages**

In [4]:
# public
import os
import numpy as np

# private
from config import *
from preprocess import make_dataset
from hyperparameter_search import grid_search
import train
from predict import future_predict
from evaluate import offline_analysis, summary_table, visualize_offline_results, map_visualize_offline_analysis

**set config**

In [5]:
cfg = config()

**preprocess**

In [6]:
# make dataset
if not os.path.exists(os.path.join(cfg.DATA_DIR, "dataset_"+cfg.PROBLEM+".npz")) or cfg.REMAKING:
    make_dataset(cfg)

# load dataset
npz = np.load(os.path.join(cfg.DATA_DIR, "dataset_"+cfg.PROBLEM+".npz"))
period_info, mesh_ids, labels, features = npz['arr_0'], npz['arr_1'], npz['arr_2'], npz['arr_3']
period_info = list(period_info)

print('problem: ', cfg.PROBLEM)
print('areas: ', cfg.area_list)
print('preriod of dataset: ', period_info)
print('length of period: ', len(period_info))
print('total mesh ids: ', mesh_ids.shape)
print('shape of total labels: ', labels.shape)
print('shape of total features: ', features.shape)

# add the dataset to config
cfg.period_info = period_info
cfg.mesh_ids = mesh_ids
cfg.labels = labels
cfg.features = features

area:  porto_velho
area:  humaita
area:  altamira
area:  vista_alegre_do_abuna
area:  novo_progresso
area:  sao_felix_do_xingu
area:  S6W57
area:  S7W57
problem:  classification
areas:  ['porto_velho', 'humaita', 'altamira', 'vista_alegre_do_abuna', 'novo_progresso', 'sao_felix_do_xingu', 'S6W57', 'S7W57']
preriod of dataset:  ['201810', '201811', '201812', '201901', '201902', '201903', '201904', '201905', '201906', '201907', '201908', '201909', '201910', '201911', '201912', '202001', '202002', '202003', '202004', '202005', '202006', '202007', '202008', '202009', '202010', '202011', '202012', '202101', '202102', '202103', '202104', '202105', '202106', '202107', '202108', '202109', '202110', '202111', '202112', '202201', '202202', '202203', '202204', '202205', '202206', '202207', '202208', '202209']
length of period:  48
total mesh ids:  (95730,)
shape of total labels:  (95730, 48)
shape of total features:  (95730, 48, 12, 3)


**hyperparameter search**

In [7]:
if not os.path.exists(os.path.join(cfg.MODEL_DIR, "best_hyperparams_"+cfg.PROBLEM+".csv")):
    print('-----------------')
    print('start hyperparameter search')
    print('-----------------')
    grid_search(cfg)
else:
    print('-----------------')
    print('already searched')
    print('-----------------')

-----------------
already searched
-----------------


**check GPU type (if you set cpu, please change your GPU type to T4 GPU)**

In [8]:
import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device) # cuda:0 is OK

cuda:0


**training with optimized hyperparameters**

In [9]:
if not os.path.exists(os.path.join(cfg.MODEL_DIR, "LSTM_"+cfg.train_end_period_list[-1]+"_"+cfg.PROBLEM+".pytorch")):
    print('---------------')
    print('start training')
    print('---------------')
    train.fit(cfg)
elif cfg.RETRAINING==True:
    print('-----------------')
    print('start retraining')
    print('-----------------')
    train.fit(cfg)
else:
    print('-----------------')
    print('already trained')
    print('-----------------')

---------------
start training
---------------
train end period:  202009
balanced_train_labels:  (25614,)
balanced_train_features:  (25614, 12, 3)


[Epoch (val) 3/100]: 100%|██████████| 81/81 [00:00<00:00, 361.88it/s]


EarlyStopping counter: 1 out of 5


[Epoch (val) 4/100]: 100%|██████████| 81/81 [00:00<00:00, 367.74it/s]


EarlyStopping counter: 2 out of 5


...

EarlyStopping counter: 3 out of 5


[Epoch (val) 10/100]: 100%|██████████| 174/174 [00:00<00:00, 619.59it/s]


EarlyStopping counter: 4 out of 5


[Epoch (val) 11/100]: 100%|██████████| 174/174 [00:00<00:00, 553.09it/s]


EarlyStopping counter: 5 out of 5


**offline evaluation**

In [10]:
if not os.path.exists(os.path.join(cfg.RESULT_DIR, "results_"+cfg.PROBLEM+".npz")):
    print('-----------------')
    print('start evaluation')
    print('-----------------')
    offline_analysis(cfg)
    summary_table(cfg)
    visualize_offline_results(cfg)
    map_visualize_offline_analysis(cfg)
elif cfg.REEVALUATION==True:
    print('-------------------')
    print('start reevaluation')
    print('-------------------')
    offline_analysis(cfg)
    summary_table(cfg)
    visualize_offline_results(cfg)
    map_visualize_offline_analysis(cfg)
else:
    print('-----------------')
    print('already evaluated')
    print('-----------------')

-----------------
start evaluation
-----------------
test period:  202010
test period:  202011
test period:  202012
test period:  202101
test period:  202102
test period:  202103
test period:  202104
test period:  202105
test period:  202106
test period:  202107
test period:  202108
test period:  202109
test period:  202110
test period:  202111
test period:  202112
test period:  202201
test period:  202202
test period:  202203
test period:  202204
test period:  202205
test period:  202206
test period:  202207
test period:  202208
test period:  202209
summary table
porto_velho
           acc    recall   precision  f1 score
LSTM_mean:  0.867   0.772     0.557       0.566
LSTM_max:   0.903   0.811     0.593       0.624
LR_mean:  0.717   0.794     0.534       0.483
LR_max:   0.752   0.818     0.555       0.530


humaita
           acc    recall   precision  f1 score
LSTM_mean:  0.893   0.756     0.542       0.544
LSTM_max:   0.950   0.800     0.612       0.653
LR_mean:  0.714   0.806     0

<Figure size 1500x1000 with 0 Axes>

<Figure size 1500x1000 with 0 Axes>

<Figure size 1500x1000 with 0 Axes>

<Figure size 1500x1000 with 0 Axes>

<Figure size 1500x1000 with 0 Axes>

<Figure size 1500x1000 with 0 Axes>

<Figure size 1500x1000 with 0 Axes>

<Figure size 1500x1000 with 0 Axes>

<Figure size 1500x1000 with 0 Axes>

**furure prediction**

In [11]:
# predict deforestation event of next month (no true values) using the latest model
if cfg.FUTURE_PREDICT == True:
    # future prediction (no true values)
    print('-------------------------')
    print(' start future prediction ')
    print('-------------------------')
    future_predict(cfg)

-------------------------
 start future prediction 
-------------------------
pred period:  202209
